#Fixed_rate_investment(정률투자법)

#Basic Setting
### Load modules

In [ ]:
import pandas as pd
import requests
from tqdm import tqdm

# Get Daiiy Prices

### KODEX 200(Daily)

In [ ]:
kodex_200 = pd.DataFrame()

kodex_url = 'https://finance.naver.com/item/sise_day.nhn?code=069500' 

for page in tqdm(range(1, 100)) : 
    pg_url = '{url}&page={page}'.format(url=kodex_url, page=page)

    pg_url = requests.get(pg_url, headers={'User-agent' : 'Mozilla/5.0'}).text
    kodex_200 = kodex_200.append(pd.read_html(pg_url)[0], ignore_index=True)



In [ ]:
kodex_200 = kodex_200.dropna()
kodex_200.columns = ['date', 'close', 'over_the_day', 'open', 'high', 'low', 'volume']
kodex_200.date = pd.to_datetime(kodex_200.date)

print('출처: ', kodex_url)
print(kodex_200.shape)
kodex_200.head()

In [ ]:
type(kodex_200.iloc[0]['date'])

In [ ]:
kodex_200

### Gov 3-year(Daily)

In [ ]:
gov_3 = pd.DataFrame()
gov_url = 'https://finance.naver.com//item/sise_day.nhn?code=114260'
for page in tqdm(range(1, 100)):
    pg_url = '{url}&page={page}'.format(url=gov_url, page=page)
    pg_url = requests.get(pg_url, headers={'User-agent' : 'Mozilla/5.0'}).text
    gov_3 = gov_3.append(pd.read_html(pg_url)[0], ignore_index=True)

In [ ]:
gov_3 = gov_3.dropna()
gov_3.columns = ['date', 'close', 'over_the_day', 'open', 'high', 'low', 'volume']
gov_3.date = pd.to_datetime(gov_3.date)
gov_3

In [ ]:
kodex_200

In [ ]:
gov_3

###Turn into Monthly DataFrame

In [ ]:
kodex_monthly = pd.DataFrame()
gov_monthly = pd.DataFrame()
for y in range(2017, 2022):
    for m in range(1, 13) : 
        kodex_monthly = kodex_monthly.append(kodex_200[(kodex_200['date'].dt.year == y) & (kodex_200['date'].dt.month == m)].head(1))
        gov_monthly =  gov_monthly.append(gov_3[(gov_3.date.dt.year == y) & (gov_3.date.dt.month == m)].head(1))


range(0,len(gov_monthly))

# list comprehension

kodex_monthly.index = [x for x in range(len(gov_monthly))]
gov_monthly.index = [x for x in range(len(gov_monthly))]

In [ ]:
kodex_monthly

In [ ]:
gov_monthly

# Backtesting

In [ ]:
# psuedo function
def backtest(beginning_asset = 100000):
    kodex_pct = 0.5
    gov_pct = 1 - kodex_pct
    asset = [beginning_asset]
# range(len(kodex_monthly)-1) 동안 backtesting을 함
    for i in range(len(kodex_monthly)-1):
        # KOSPI
        kodex_shares = (beginning_asset * kodex_pct) / kodex_monthly.iloc[i].close # t 시점
        kodex_ending_value = kodex_shares * kodex_monthly.iloc[i+1].close # t+1 시점
        

        # Gov 3-year
        gov_shares = (beginning_asset * gov_pct) / gov_monthly.iloc[i].close # t 시점
        gov_ending_value = gov_shares * gov_monthly.iloc[i+1].close  # t+1 시점

        # rebalancing
        total_ending_value = kodex_ending_value + gov_ending_value

        asset.append(total_ending_value)

        beginning_asset = total_ending_value
    # assetinsert(인덱스, 값)
    # asset.insert(0, 100000)

    return asset

In [ ]:
'''
|1       |2  = t                                              |3 = t+1| ..... |-|-|49|
|k 주식수|k 주식수 x 2회차의 가격 = 2회차의 시점의 k value
|g 주식수|g 주식수 x 2회차의 가격 = 2회차의 시점의 g value
|.       |                                         total value = begging_asset
'''

In [ ]:
result = backtest()


In [ ]:
back_test = {
    'date' : kodex_monthly['date'], # 날짜 
    'backtest' : result,
    'KODEX_200' : kodex_monthly[ 'close'] / kodex_monthly.iloc[0].close * 100000, # 코스피 벤치마크
    'GOV_3Y' : gov_monthly['close'] / gov_monthly.iloc[0].close * 100000 # 국고체3년물 벤치마크
}

In [ ]:
back_test = pd.DataFrame(back_test)
back_test

#Visualization

In [ ]:
# magic code
%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
'''
6[show()]
5[격자(grid) 얹기]
4[범례 얹기]
3[그래프 2 얹기]
2[그래프 1 얹기]
1[캔버스 (바닥)]
'''

In [ ]:
plt.figure(figsize=(18, 4))
plt.plot(back_test.date,back_test.backtest, label='Backtest') # 라인 그래프
plt.plot(back_test.date, back_test.KODEX_200, label='KODEX_200')
plt.plot(back_test.date, back_test.GOV_3Y, label='GOV_3Y')

plt.legend(loc='best') # 범례

plt.title('Backtesting Portfolio & Benchmark', fontsize=25)
plt.xlabel('Year', fontsize=18)
plt.ylabel('Asset Amount', fontsize=18 )


plt.grid()
plt.show()

# Compute CAGR
(compounded annual growth return)

### backtesting CAGR

In [ ]:
begin, end = result[0], result[-1]

cagr = ((end/begin)**(1/len(set(kodex_200.date.dt.year))) -1 ) * 100
print('%.2f'%cagr,'%')

###KODEX 200 CAGR

In [ ]:
begin, end = kodex_monthly.iloc[0].close, kodex_monthly.iloc[-1].close
cagr = ((end/begin)**(1/len(set(kodex_200.date.dt.year))) -1 ) * 100
print('%.2f'%cagr,'%')

###GOV 3Y CAGR

In [ ]:
begin, end = gov_monthly.iloc[0].close, gov_monthly.iloc[-1].close
cagr = ((end/begin)**(1/len(set(kodex_200.date.dt.year))) -1 ) * 100
print('%.2f'%cagr,'%')